In [1]:
import pandas as pd
pd.options.plotting.backend = "plotly" #interactive plots will be useful in this context
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import gensim
from sentence_transformers import SentenceTransformer, util
import pickle
import torch
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

C:\Users\BothmK01\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BothmK01\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BothmK01\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [142]:
df = pd.read_csv('../datasets/df_full.csv')
df_r=pd.read_csv('../datasets/df_k31_k39.csv')
df_cv = pd.read_csv('../datasets/df_cv.csv')
df_handbook = pd.read_csv('../datasets/df_handbook_DS_60.csv')
k39_umap = pickle.load(open('../Model_Selection/k_39_umap', 'rb'))
k39_full = pickle.load(open('../Model_Selection/k_39_full', 'rb'))
k31_umap = pickle.load(open('../Model_Selection/k_31_umap', 'rb'))
k31_full = pickle.load(open('../Model_Selection/k_31_full', 'rb'))

In [3]:
list_of_vec = []
job_list = []
for index, row in df_r.iterrows():
    job = row['url']
    if job not in job_list:
        job_list.append(job)
print(len(job_list))
for job in job_list:
    df_r_job = df_r[df_r.url == job]
        
    cluster_list = []
    cluster_vec = []

    for i, row_r in df_r_job.iterrows():
        cluster = row_r['cluster_k31_full']
        if cluster not in cluster_list:
            cluster_list.append(cluster)
            
    cluster_vec.append(job)
    for i in range(0,31):
        if(i in cluster_list):
            cluster_vec.append(1)
        else:
            cluster_vec.append(0)
    list_of_vec.append(cluster_vec)

cols = []
cols.append('job_link')
for i in range(0,31):
    cols.append(str(i))
df_clusters = pd.DataFrame(list_of_vec, columns = cols)

1086


In [4]:
df_clusters

,job_link,0,1,2,3,4,5,6,7,8,...,21,22,23,24,25,26,27,28,29,30
0,https://se.indeed.com/viewjob?jk=b0669075c820856d,0,0,0,0,0,0,1,0,0,...,1,1,0,1,1,0,0,0,0,0
1,https://se.indeed.com/viewjob?jk=dbfe7d7c087065d7,0,0,0,0,0,0,0,0,1,...,1,0,1,0,0,1,0,0,0,0
2,https://se.indeed.com/viewjob?jk=841a8fdd1bb00f2f,0,1,1,0,1,0,1,0,0,...,0,0,1,0,0,0,1,0,1,0
3,https://se.indeed.com/viewjob?jk=88c25a48d2da8a7a,1,0,1,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1
4,https://se.indeed.com/viewjob?jk=1b4b8ee005f06350,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,https://www.indeed.com/viewjob?jk=e6fee8e1f133...,0,0,0,0,0,0,0,1,1,...,0,0,1,0,1,0,0,0,1,1
1082,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,0,0,0,0,0,0,0,1,1,...,1,1,0,1,1,1,0,0,0,1
1083,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1
1084,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,0,0,0,0,0,0,1,0,1,...,0,0,1,1,0,0,0,0,0,1


In [5]:
from collections import Counter
def get_bigrams(requirement_list):
    list = []
    for requirement in requirement_list:
        for word in requirement.split():
            word = word.lower()
            if (len(word) >1) & (word not in stop_words):
                list.append(word)
    
    bigrams = zip(list,list[1:])
    #print(list)
    bigrams_count = Counter(bigrams)
    return(bigrams_count)

In [6]:
def get_label(cluster):#model needs to be incorperated if plotting different models
    df_r_cluster = df_r[df_r.cluster_k31_full == cluster]
    cluster_requirement_list = []
    for i, row in df_r_cluster.iterrows():
        cluster_requirement_list.append(row['requirement'])
    tuple = get_bigrams(cluster_requirement_list).most_common(3)
    label = ""
    for bigram in range(3):   
        for word in tuple[bigram][0]:
            label+=word+" "
        label=label[:-1]
        label+=", "
    label = label[:-2]
    label = label.replace(",,",",")
    label = "Cluster "+str(cluster)+": "+label
    return(label)
get_label(1)

'Cluster 1: knowledge sql, relational databases, sql queries'

In [168]:
x = []
y = []
for i in range(0,len(k31_full.cluster_centers_)):
    y.append(get_label(i))
    x.append(df_clusters[str(i)].mean())
sorted = list(zip(x,y))
sorted.sort()
x=[]
y=[]
for tuple in sorted:
    x.append(tuple[0])
    y.append(tuple[1])

fig = go.Figure([go.Bar(x=x, y=y,orientation='h')])
fig.update_layout(height=800, \
                  title = '3 most common bigrams of each cluster and relative cluster precense in job postings',\
                  barmode='stack',\
                  xaxis_title="Relative Cluster Presence in Job Postings",\
                  yaxis_title="3 most common Bigrams in Cluster",)
fig.show()

In [8]:
cv_koen = ['Data Driven Process Technologist',\
         'Partly responsible for product quality and technological yield of cheese production process',\
         'By Data Driven approach I automated 75/% of my original role and realized a yield improvement of > €500.000 / year',\
         'Responsible for setting up and maintaining on-the-job training programs and masterclasses for on-boarding and up-skilling of operators',\
         'After working as an operator I took advantage of a growth opportunity to lead a team of 7 operators in 3 production departments',\
         'Training & Education specialist production',\
         'Development of physics lab excercise program',\
         'Mentor of freshman students post-secondary vocational education',\
         'Data Science (MSc) IU International University',\
         'Environment: Python', 'SQL', 'Visual Basic, M, Dax',\
         'Jupyter Notebook', 'Dataiku Data Science Studio', 'Docker',\
         'Dashboarding: Power BI (expert) / Tibco Spotfire',\
         'English', 'Dutch']

In [183]:
def get_df(cv, plot):
    df_sim = pd.DataFrame()
    df_sim['cluster'] = range(len(k31_full.cluster_centers_))
    
    labels = []
    presence = []
    
    for i in range(0,len(k31_full.cluster_centers_)):
        labels.append(get_label(i))
        presence.append(df_clusters[str(i)].mean())
    
    model = 'all-distilroberta-v1'
    model = SentenceTransformer(model)
    embeddings_cv = model.encode(cv)
    embeddings_f = embeddings_cv.astype(float)
    clusters_cv = k31_full.predict(embeddings_f)
    clusters_cv_l  = clusters_cv.tolist()
    
    cv_scores = []
    for i, cluster in enumerate(clusters_cv):
        cv_scores.append(util.pytorch_cos_sim(k31_full.cluster_centers_[cluster], embeddings_f[i]).item())
    
    scores = []
    for cluster in range(len(k31_full.cluster_centers_)):
        if cluster not in clusters_cv_l:
            scores.append(0)
        else:
            score = 0
            indexes = np.where(clusters_cv==cluster)[0]
            for i in indexes:
                if cv_scores[i] > score:
                    score = cv_scores[i]
            scores.append(score)   
    
    df_sim['score'] = scores
    df_sim['presence'] = presence
    df_sim['labels'] = labels
    
    df_sim['CV_similarity'] = df_sim['presence']*df_sim['score']
    df_sim['Presence_Cluster_in_Job_Postings'] = df_sim['presence']-df_sim['CV_similarity']
    df_sim = df_sim.sort_values('presence')
    
    if plot:
        fig = px.bar(df_sim, y="labels", x=["CV_similarity","Presence_Cluster_in_Job_Postings"], hover_data = ['presence'])
        fig.update_layout(height=800, \
                          title = 'Author\'s CV similarity to requirement clusters in context of relative cluster presence',\
                          barmode='stack', \
                          yaxis_title="3 most common Bigrams in Cluster",\
                          xaxis_title="CV Similarity to Cluster")
        fig.show()
        fig = px.bar(df_sim, y="labels", x=["CV_similarity","Presence_Cluster_in_Job_Postings"], hover_data = ['presence'])
        fig.show()
    return df_sim
    
df_sim = get_df(cv_koen, plot = True)

In [21]:
df_cv.head()

,Unnamed: 0,id,title,skill,cluster,embedding
0,0,0,Data Scientist/Analytics Consultant,"design sql, r & python scripts to directly qu...",11,"[0.020906170830130577, -0.02053937129676342, -..."
1,1,0,Data Scientist/Analytics Consultant,aid in the development of predictive modeling...,6,"[0.0457504577934742, -0.03818187490105629, -0...."
2,2,0,Data Scientist/Analytics Consultant,test model fit using cross-validation and boo...,19,"[-0.011868440546095371, -0.0615849532186985, -..."
3,3,0,Data Scientist/Analytics Consultant,develop in an scrum environment with team mem...,16,"[0.04099218547344208, -0.02379242517054081, 0...."
4,4,0,Data Scientist/Analytics Consultant,"model data using pandas/r dataframes, powerpi...",19,"[-0.0024511003866791725, -0.02079622820019722,..."


In [52]:
df_candidate = df_cv[df_cv['id']==8]
skills = df_candidate['skill'].to_list()

In [50]:
df_sim = plot_cv(skills)
df_sim['CV_similarity'].sum()/df_sim['presence'].sum()

0.22641564039781853

In [68]:
df_cv.head()

,Unnamed: 0,id,title,skill,cluster,embedding
0,0,0,Data Scientist/Analytics Consultant,"design sql, r & python scripts to directly qu...",11,"[0.020906170830130577, -0.02053937129676342, -..."
1,1,0,Data Scientist/Analytics Consultant,aid in the development of predictive modeling...,6,"[0.045750461518764496, -0.03818187490105629, -..."
2,2,0,Data Scientist/Analytics Consultant,test model fit using cross-validation and boo...,19,"[-0.011868440546095371, -0.0615849532186985, -..."
3,3,0,Data Scientist/Analytics Consultant,develop in an scrum environment with team mem...,16,"[0.04099218547344208, -0.02379242517054081, 0...."
4,4,0,Data Scientist/Analytics Consultant,"model data using pandas/r dataframes, powerpi...",19,"[-0.0024511003866791725, -0.02079622820019722,..."


In [102]:
df_sim_all = pd.DataFrame()

for i in df_cv['id'].unique():
    df_ = df_cv[df_cv['id']==i]
    skills = df_['skill'].to_list()
    if len(df_sim_all)==0:
        df_sim_all = get_df(skills,False)
    else:
        df_sim_all = df_sim_all.append(get_df(skills,False),ignore_index=True)
    print(i)

0
62
1
93
2
124
3
155
4
186
5
217
6
248
7
279
8
310
9
341
10
372
11
403
12
434
13
465
14
496
15
527
16
558
17
589
18
620
19
651
20
682
21
713
22
744
23
775
24
806
25
837
26
868
27
899
28
930
29
961
30
992
31
1023
32
1054
33
1085
34
1116
35
1147
36
1178
37
1209
38
1240
39
1271
40
1302
41
1333
42
1364
43
1395
44
1426
45
1457
46
1488
47
1519
48
1550
49
1581
50
1612
51
1643
52
1674
53
1705
54
1736
55
1767
56
1798
57
1829
58
1860
59
1891
60
1922
61
1953
62
1984
63
2015
64
2046
65
2077
66


In [103]:
len(df_sim_all)

2077

In [105]:
df_sim_all.to_csv('../datasets/df_sim_cv_all.csv')

In [118]:
df_sim_all[df_sim_all['cluster']==1]

,cluster,score,presence,labels,CV_similarity,Presence_Cluster_in_Job_Postings
4,1,0.000000,0.237569,"Cluster 1: knowledge sql, relational databases...",0.000000,0.237569
35,1,0.567574,0.237569,"Cluster 1: knowledge sql, relational databases...",0.134838,0.102731
66,1,0.597215,0.237569,"Cluster 1: knowledge sql, relational databases...",0.141880,0.095689
97,1,0.642046,0.237569,"Cluster 1: knowledge sql, relational databases...",0.152530,0.085039
128,1,0.000000,0.237569,"Cluster 1: knowledge sql, relational databases...",0.000000,0.237569
...,...,...,...,...,...,...
1926,1,0.549192,0.237569,"Cluster 1: knowledge sql, relational databases...",0.130471,0.107098
1957,1,0.000000,0.237569,"Cluster 1: knowledge sql, relational databases...",0.000000,0.237569
1988,1,0.512586,0.237569,"Cluster 1: knowledge sql, relational databases...",0.121775,0.115795
2019,1,0.000000,0.237569,"Cluster 1: knowledge sql, relational databases...",0.000000,0.237569


In [127]:
print(df_sim_all.columns)

Index(['cluster', 'score', 'presence', 'labels', 'CV_similarity',
       'Presence_Cluster_in_Job_Postings'],
      dtype='object')


In [141]:
df_sim_avg = pd.DataFrame(columns = df_sim_all.columns)
for cluster in range(len(k31_full.cluster_centers_)):
    df_=df_sim_all[df_sim_all['cluster']==cluster]
    df_ = df_.mean()
    df_['labels'] = get_label(cluster)
    df_sim_avg = df_sim_avg.append(df_, ignore_index = True)
df_sim_avg = df_sim_avg.sort_values('presence')
df_sim_avg.mean()

cluster                             15.000000
score                                0.331117
presence                             0.346938
CV_similarity                        0.119855
Presence_Cluster_in_Job_Postings     0.227082
dtype: float64

In [187]:
fig = px.bar(df_sim_avg, y="labels", x=["CV_similarity","Presence_Cluster_in_Job_Postings"], hover_data = ['presence'])
fig.update_layout(height=800, \
                          title = 'Mean similarity of 65 Data Scientist CV\'s to requirement clusters in context of relative cluster presence',\
                          barmode='stack',\
                          yaxis_title="3 most common Bigrams in Cluster",\
                          xaxis_title="CV Similarity to Cluster")
fig.show()

In [145]:
df_handbook = get_df(df_handbook['objective'].to_list(),plot = True)

In [147]:
handbook_objective_similarity = df_handbook['CV_similarity'].to_list()

In [148]:
df_sim_avg['handbook_similarity'] = handbook_objective_similarity

In [151]:
df_sim_avg

,cluster,score,presence,labels,CV_similarity,Presence_Cluster_in_Job_Postings,handbook_similarity
24,24.0,0.389511,0.149171,"Cluster 24: deep learning, machine learning, e...",0.058104,0.091067,0.096440
20,20.0,0.161375,0.170350,"Cluster 20: experience working, single cell, n...",0.027490,0.142860,0.000000
3,3.0,0.010119,0.188766,"Cluster 3: years experience, related field, ye...",0.001910,0.186856,0.000000
27,27.0,0.393819,0.232965,"Cluster 27: experience cloud, google cloud, cl...",0.091746,0.141219,0.000000
1,1.0,0.495865,0.237569,"Cluster 1: knowledge sql, relational databases...",0.117802,0.119767,0.157216
5,5.0,0.056899,0.237569,"Cluster 5: track record, research projects, sc...",0.013517,0.224052,0.114492
14,14.0,0.528157,0.244015,"Cluster 14: data visualization, visualization ...",0.128878,0.115137,0.177415
19,19.0,0.443944,0.274401,"Cluster 19: predictive models, machine learnin...",0.121819,0.152583,0.192142
2,2.0,0.087041,0.277164,"Cluster 2: ability work, work independently, f...",0.024125,0.253039,0.000000
8,8.0,0.521661,0.291897,"Cluster 8: big data, experience working, exper...",0.152271,0.139626,0.169745


In [190]:
fig = px.bar(df_sim_avg, y="labels", x=["handbook_similarity","CV_similarity","presence"],\
             color_discrete_sequence =['green','blue','red'])
fig.update_layout(height=850, \
                          title = 'Similarity of CV\'s and Learning Curriculum to Requirements in Job Postings',\
                          barmode='group',\
                          yaxis_title="3 most common Bigrams in Cluster",\
                          xaxis_title="CV Similarity to Cluster")
fig.show()

In [191]:
for i in range(31):
    print(get_label(i))

Cluster 0: team player, ability work, team members
Cluster 1: knowledge sql, relational databases, sql queries
Cluster 2: ability work, work independently, fast-paced environment
Cluster 3: years experience, related field, years relevant
Cluster 4: learn new, new technologies, eager learn
Cluster 5: track record, research projects, scientific technical
Cluster 6: machine learning, learning models, learning algorithms
Cluster 7: communication skills, excellent communication, skills excellent
Cluster 8: big data, experience working, experience big
Cluster 9: attention detail, natural language, social media
Cluster 10: experience working, software development, project management
Cluster 11: data sets, data analysis, data sources
Cluster 12: machine learning, learning techniques, machine learning,
Cluster 13: data science, data scientists, data scientist
Cluster 14: data visualization, visualization tools, experience data
Cluster 15: years experience, experience data, data science
Cluster 